In [1]:
from datacleanv2 import *
from SetRNN import *
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
from collections import Counter # 用于统计计数的工具
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import time # 用于计时
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils # 用于处理变长序列，如填充和打包
from torch.utils.data import Dataset, DataLoader # PyTorch 数据加载工具
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm # 进度条库，使用 tqdm.tqdm
import random
import copy # 用于复制模型参数或列表
import matplotlib.pyplot as plt # 用于绘图
import seaborn as sns # 用于更美观的统计图，特别是热力图
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EARLY_ITER_BATCH_THRESHOLD = 3 # 在前 3 轮迭代中使用部分批次 (适应总迭代 10)
EARLY_ITER_BATCH_PERCENT = 0.3

# 超参数和常量定义
NUM_MAIN_MODELS = 3 # 主要的聚类模型数量
NUM_COMBINED_SETTINGS = 13 # combined_setting 的总类别数 (0-124)
EMBEDDING_DIM = 8 # combined_setting 的嵌入向量维度，可调整
HIDDEN_SIZE = 64   # RNN 隐藏层大小，可调整
NUM_RNN_LAYERS = 2 # RNN 层数
# 注意: TIME_LOSS_SCALER 可能需要根据实际 delta_t 的规模重新调整
TIME_LOSS_SCALER = 1 # time delta_t MSE 损失的缩放因子，需要根据实际损失值大小调整
TOTAL_EM_ITERATIONS = 10 # EM 迭代总次数 (根据要求修改为 10)
CONVERGENCE_THRESHOLD = 0.05 # 收敛阈值，分配改变的序列比例低于此值时停止 (5%)

# 干扰项处理参数
NUM_RAND_SEQUENCES = 250 # 干扰项的已知数量
INTERFERENCE_CLUSTER_LABEL = 3 # 将干扰项分配到的簇的索引 (0, 1, 2 是主簇，3 是干扰簇)
INTERFERENCE_DETECTION_START_ITER = 2 # 从第 5 轮迭代 (索引 4) 的 E 步开始检测干扰项
# 检测干扰项的高损失阈值：需要根据训练中观察到的损失值范围来调整
# 如果一个序列在所有模型上的平均损失超过这个阈值，则可能被认为是干扰项。
# ！！！重要参数，需要根据实际运行观察的损失值调整！！！
# 在模拟数据上运行一次，观察损失值的分布，尤其是 rand_label 序列的损失。
HIGH_AVG_LOSS_THRESHOLD = 0.5 ## <--- !!! 初始值，请务必根据实际情况调整 !!!

# M 步训练参数 (每个 EM 迭代中的训练 epochs)
# epochs 计划表：根据迭代次数使用不同数量的 epochs
EPOCHS=20
BATCH_SIZE = 32 # M 步训练时的批次大小
# 在早期迭代中是否只使用部分批次来加速训练
EARLY_ITER_BATCH_THRESHOLD = 3 # 在前 3 轮迭代中使用部分批次 (适应总迭代 10)
EARLY_ITER_BATCH_PERCENT = 0.3 # 在启用部分批次训练时使用的批次比例 (30%)

In [2]:
def create_time_series_with_result(result: list[pd.DataFrame]) -> list[pd.DataFrame]:
    """
    将子 DataFrame 列表中的每个元素转换为一个包含时间序列结果的列表。

    Args:
        result: 包含 Pandas DataFrame 的列表。

    Returns:
        一个最终列表，其中每个元素都是一个包含时间序列 的DataFrame的列表。
    """
    final_list = []
    for df in result:
        if not df.empty:
            # 1. 提取时间序列数据
            time_series_df = df[['time', 'event_value']].copy()
            time_series_df = time_series_df.rename(columns = {"event_value":"combined_setting"})
            final_list.append(time_series_df)
        else:
            final_list.append(pd.DataFrame(columns=['time','combined_setting'])) # 处理空 DataFrame

    return final_list

def encode_event_values(dataframes_list: list[pd.DataFrame]) -> list[pd.DataFrame]:
    """
    将每个dataframe中的combined_setting列从字符串映射为0-12的数字编码
    
    Args:
        dataframes_list: 包含多个dataframe的列表，每个dataframe有time和combined_setting两列
        
    Returns:
        转换后的dataframe列表，combined_setting列变为数字编码
    """
    # 定义映射字典
    event_mapping = {
        'Buy': 0,
        'Cancel': 1,
        'city_subway': 2,
        'concession': 3,
        'country_trains': 4,
        'daily': 5,
        'full_fare': 6,
        'individual': 7,
        'trip_1': 8,
        'trip_2': 9,
        'trip_3': 10,
        'trip_4': 11,
        'trip_5': 12
    }

    # 处理每个dataframe
    encoded_dataframes = []
    
    for i, df in enumerate(dataframes_list):
        # 检查dataframe结构
        if not {'time', 'combined_setting'}.issubset(df.columns):
            raise ValueError(f"第{i}个dataframe缺少time或combined_setting列")
        # 复制dataframe以避免修改原始数据
        df_encoded = df.copy()
        # 映射combined_setting列
        df_encoded['combined_setting'] = df_encoded['combined_setting'].map(event_mapping)
        encoded_dataframes.append(df_encoded)
    
    return encoded_dataframes


In [3]:
df,mata = pyreadstat.read_sav(r"E:\复旦大学\研一上\科研\评分剪枝算法\数据\tickets\CBA_cp038q01_logs12_SPSS.sav")
result =[item[1:-1] for item in split_strict_paired_events(df)]
final_result_list_raw=create_time_series_with_result(result)
transformed_list = encode_event_values(final_result_list_raw)
filtered_list = [[df,''] for df in transformed_list if len(df)>=3]
#加一个''作为response占位符

In [4]:
transformed_list=filtered_list

# 训练一个大RNN

In [5]:
def train_model(model, dataloader, optimizer, time_criterion, setting_criterion, epochs, time_scaler, iteration_num,writer):
    model.train() # 设置模型为训练模式
    total_batches = len(dataloader)
    # 根据迭代次数决定使用的批次数量
    if iteration_num < EARLY_ITER_BATCH_THRESHOLD:
        batches_to_use = max(1, int(total_batches * EARLY_ITER_BATCH_PERCENT))
    else:
        batches_to_use = total_batches

    # 使用 tqdm 显示 epoch 进度，使用传入的 model_idx
    epoch_tqdm = tqdm(range(epochs), desc=f"迭代 {iteration_num} (模型 ) 训练", leave=False)
    for epoch in epoch_tqdm:
        total_epoch_loss = 0
        total_time_loss = 0
        total_setting_loss = 0
        batch_count = 0
        # 使用 tqdm 显示批次进度,dataloader每次迭代返回一个batch（4条序列）和一条长度表
        batch_tqdm = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for batch_data in batch_tqdm:
            if batch_data is None: continue # 跳过空批次

            # 从 collate_fn 获取批次数据,填充和对t作差分是在collate_fn中完成,将setting嵌入是在rnn中完成
            delta_t_inputs, setting_inputs, delta_t_targets, setting_targets, lengths = batch_data

            optimizer.zero_grad() # 清零梯度

            # 前向传播
            # 模型现在输出的是 predicted_next_delta_t 和 predicted_next_setting_logits
            predicted_next_delta_t, predicted_next_setting_logits, _ = model(delta_t_inputs, setting_inputs, lengths)

            # 计算损失
            # predicted_next_delta_t 形状: (batch_size, seq_len)
            # delta_t_targets 形状: (batch_size, seq_len)
            time_loss = time_criterion(predicted_next_delta_t, delta_t_targets)

            # predicted_next_setting_logits 形状: (batch_size, seq_len, num_categories)
            # setting_targets 形状: (batch_size, seq_len)
            # 需要调整 logits 的维度到 (batch_size, num_categories, seq_len)
            setting_loss = setting_criterion(predicted_next_setting_logits.permute(0, 2, 1), setting_targets)

            # 计算总损失，并应用时间损失缩放因子
            loss = time_loss * time_scaler + setting_loss

            # 反向传播和优化
            loss.backward()
            # 可选：梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_epoch_loss += loss.item() # 计算epoch累计损失
            total_time_loss += time_loss.item()
            total_setting_loss += setting_loss.item()
            batch_count += 1

            # 在早期迭代中，只训练部分批次
            if batch_count >= batches_to_use:
                 break

            # 更新批次进度条的后缀信息（可选）
            batch_tqdm.set_postfix(loss=loss.item())

        final_epoch_avg_loss = total_epoch_loss / batch_count 
        final_epoch_avg_time_loss = total_time_loss / batch_count
        final_epoch_avg_setting_loss = total_setting_loss / batch_count
        writer.add_scalar("Loss/Total_train",final_epoch_avg_loss,epoch+1) # 记录每个 epoch 的平均损失到 TensorBoard
        writer.add_scalar('Loss/Time_Loss_Train', final_epoch_avg_time_loss, epoch)
        writer.add_scalar('Loss/Setting_Loss_Train', final_epoch_avg_setting_loss, epoch)
        
        # 更新 epoch 进度条的后缀信息（可选）
        # 注意：这里显示的是最后一个批次的损失，不是平均损失
        epoch_tqdm.set_postfix(last_batch_loss=loss.item())


In [6]:
def evaluate_sequence_loss(model, time_seq_full, setting_seq_full, time_criterion, setting_criterion, time_scaler):
    model.eval() # 设置模型为评估模式
    with torch.no_grad(): # 禁用梯度计算

        seq_len = time_seq_full.size(0)
        # 序列长度小于 3 无法构建输入和目标序列 (长度 original_length - 2)
        if seq_len < 3:
            # 返回无穷大，表示无法计算有效损失，在比较时会被排除
            return float('inf')

        # --- 构建输入序列 (当前 delta_t 和 setting) 和目标序列 (下一个 delta_t 和 setting) ---
        # 它们都对应原始序列长度 - 2 的部分

        # delta_t 输入: time[i+1] - time[i] for i from 0 to seq_len - 3
        delta_t_inputs_sliced = (time_seq_full[1:-1] - time_seq_full[:-2]).unsqueeze(0).to(device) # 形状: (1, seq_len - 2)
        # setting 输入: setting[i] for i from 0 to seq_len - 3
        setting_inputs_sliced = setting_seq_full[:-2].unsqueeze(0).to(device) # 形状: (1, seq_len - 2)

        # delta_t 目标: time[i+2] - time[i+1] for i from 0 to seq_len - 3
        delta_t_targets_sliced = (time_seq_full[2:] - time_seq_full[1:-1]).unsqueeze(0).to(device) # 形状: (1, seq_len - 2)
        # setting 目标: setting[i+1] for i from 0 to seq_len - 3
        setting_targets_sliced = setting_seq_full[1:-1].unsqueeze(0).to(device) # 形状: (1, seq_len - 2)

        # 输入序列的实际长度
        eval_input_len = seq_len - 2 # 有效的预测步数
        lengths = torch.tensor([eval_input_len]) # 保持在 CPU

        # 前向传播，计算整个序列的预测结果 (长度为 eval_input_len)
        predicted_next_delta_t, predicted_next_setting_logits, _ = model(delta_t_inputs_sliced, setting_inputs_sliced, lengths)

        # 计算整个序列的损失 (注意：损失函数如 MSELoss 和 CrossEntropyLoss 默认计算的是批次和序列长度上的平均)
        # 但在这里我们处理的是单个序列 (batch_size = 1)，并且使用了 pack_padded_sequence，
        # PyTorch 会确保损失计算只在有效长度上进行。
        # 所以 time_criterion(predicted_next_delta_t, delta_t_targets_sliced) 计算的是 batch 和 有效长度上的平均损失。
        # setting_criterion(...) 也类似。

        time_loss = time_criterion(predicted_next_delta_t, delta_t_targets_sliced)
        setting_loss = setting_criterion(predicted_next_setting_logits.permute(0, 2, 1), setting_targets_sliced)

        # 计算总损失 (按步加权平均)
        total_loss_per_step = time_loss * time_scaler + setting_loss

        # total_loss_per_step 现在已经是每个预测步的平均损失了 (因为 MSELoss/CrossEntropyLoss 默认对 batch 和序列长度取平均)
        # 但是 pack_padded_sequence 的行为可能会影响这个平均，为了安全和明确，我们还是按总损失再除以步数
        # 更稳妥的方法是使用 reduction='sum' 然后手动除以有效步数
        time_criterion_sum = nn.MSELoss(reduction='sum')
        setting_criterion_sum = nn.CrossEntropyLoss(reduction='sum')

        time_loss_sum = time_criterion_sum(predicted_next_delta_t, delta_t_targets_sliced)
        setting_loss_sum = setting_criterion_sum(predicted_next_setting_logits.permute(0, 2, 1), setting_targets_sliced)

        total_sum_loss = time_loss_sum * time_scaler + setting_loss_sum
    
    
        # 计算 **平均** 损失：总和损失除以有效预测步数
        average_loss_per_step = total_sum_loss / eval_input_len
        average_set_loss_per_step = setting_loss_sum / eval_input_len


        # 返回标量平均损失值
        return average_loss_per_step.item(),average_set_loss_per_step.item()

In [7]:
def run_rnn_optimize(transformed_list,  embedding_dim, hidden_size, num_rnn_layers,
                       num_categories, time_scaler, total_iterations=10, 
                       epochs=20,batch_size=32):
    # 使用当前时间创建唯一的日志目录，以区分不同的训练运行
    log_dir = f"runs/rnn_training_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir)
    print(f"TensorBoard 日志将保存在: {log_dir}")
          
    total_sequences = len(transformed_list)
    print(f"开始基于 RNN 的聚类，共有 {total_sequences} 条序列，聚成一类")

    # 1. 初始化
    # 创建一个RNN 模型
    model = SettingPredictorRNN(embedding_dim, hidden_size, num_rnn_layers, num_categories).to(device) 
    #所有序列均长度大于3

    # 定义损失函数 (用于 E 步计算单序列损失，使用 reduction='sum')
    time_criterion_sum = nn.MSELoss(reduction='sum')
    setting_criterion_sum = nn.CrossEntropyLoss(reduction='sum')
    # 定义损失函数 (用于 M 步训练批次，使用默认 reduction='mean')
    time_criterion_mean = nn.MSELoss()
    setting_criterion_mean = nn.CrossEntropyLoss()

    # active_indices: 获取当前仍在参与聚类的主簇序列的原始索引 (即 current_assignments != interference_cluster_label 的序列)
    active_indices_mask = np.array([len(item[0]) for item in transformed_list]) >= 3 
    active_indices = range(len(transformed_list))
    current_total_active_sequences = len(active_indices)

    assigned_indices_in_active = active_indices

  
    assigned_dataset = SequenceDataset([transformed_list[i][0] for i in assigned_indices_in_active])
    # collate_fn 将过滤掉长度不足 3 的序列，DataLoader 会处理批次和填充
    assigned_dataloader = DataLoader(assigned_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=0)

    # 只有当 DataLoader 不为空时才训练 (即存在长度 >= 3 的序列)
    if len(assigned_dataloader) > 0:
        # 为当前模型创建一个优化器
        optimizer = optim.Adam(model.parameters())

        # 调用训练函数训练当前模型，传入模型索引，使用 mean reduction 的损失函数
        train_model(model,assigned_dataloader, optimizer,
                time_criterion_mean, setting_criterion_mean, epochs,
                time_scaler, total_iterations,writer)
        
    writer.close() # 关闭 TensorBoard 日志记录器
    return model # 返回 models

In [8]:
transformed_list

[[     time  combined_setting
  1  1195.6                 4
  2  1202.8                 6
  3  1217.2                 5
  4  1223.7                 0,
  ''],
 [      time  combined_setting
  7   1004.7                 4
  8   1010.6                 6
  9   1017.5                 7
  10  1027.0                 0,
  ''],
 [     time  combined_setting
  13  124.8                 4
  14  127.1                 1
  15  127.9                 4
  16  128.6                 1
  17  128.0                 2
  18  132.9                 3
  19  139.7                 7
  20  142.2                 8
  21  144.4                 0,
  ''],
 [     time  combined_setting
  24  171.0                 4
  25  173.8                 6
  26  176.2                 7
  27  196.4                 0,
  ''],
 [     time  combined_setting
  30  289.1                 4
  31  294.4                 6
  32  296.7                 7
  33  299.0                11
  34  310.5                 0,
  ''],
 [     time  combined_set

In [9]:
final_model = run_rnn_optimize(transformed_list,  EMBEDDING_DIM, HIDDEN_SIZE, NUM_RNN_LAYERS,
                       NUM_COMBINED_SETTINGS, TIME_LOSS_SCALER, TOTAL_EM_ITERATIONS,EPOCHS,BATCH_SIZE)

TensorBoard 日志将保存在: runs/rnn_training_20250908-215425
开始基于 RNN 的聚类，共有 31322 条序列，聚成一类


迭代 10 (模型 ) 训练:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 2/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 3/20:   0%|          | 0/979 [00:00<?, ?it/s]

...

Epoch 16/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 17/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 18/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 19/20:   0%|          | 0/979 [00:00<?, ?it/s]

Epoch 20/20:   0%|          | 0/979 [00:00<?, ?it/s]

In [10]:
torch.save(final_model,"TotalModel_tickets.pth")